In [17]:
import pandas as pd
import numpy as np
import gc

# data preprocessing

In [2]:
# 读取CSV文件
data = pd.read_csv("data/demo12W_30.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39979422 entries, 0 to 39979421
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   pid                 int64  
 1   date                int64  
 2   move_frequency      float64
 3   move_distance       float64
 4   trip_time           float64
 5   home_time           float64
 6   norm_sum            float64
 7   norm_mean           float64
 8   norm_max            float64
 9   norm_home_activity  float64
 10  norm_home_base      float64
 11  group_year          int64  
 12  voluntary           int64  
 13  gender              int64  
 14  age                 int64  
 15  income              float64
dtypes: float64(10), int64(6)
memory usage: 4.8 GB


In [3]:
# interpolation
# 选择日期在20191201到20191231之间的行，并删除重复的行
data_comp = data[data['date'].between(20191201, 20191231)].drop_duplicates(subset=['pid', 'date'])
data = data[~data['date'].between(20191201, 20191231)]
data_comp['group_year'] = 1

# 合并数据
data = pd.concat([data, data_comp], axis=0).sort_values(by=['pid', 'date'])

# NOTE: `na_seadec`是R中的特定函数，需要Python实现或找到相应的Python替代。
# 例如，下面使用pandas的interpolate函数作为替代。
data['norm_mean'] = data.groupby(['pid', 'group_year'])['norm_mean'].transform(lambda x: x.interpolate())

In [4]:
# load weather data
precip = pd.read_csv("data/precipitation.csv")
precip.head()

,date,precip
0,20190101,0.0
1,20190102,0.1
2,20190103,0.0
3,20190104,0.1
4,20190105,0.0


In [5]:
# Adding factors
# Using the query function to filter data
data = data.query('date < 20191201 or date > 20191231')

# Convert date to string then to datetime object and create weekday column
data['datestamp'] = pd.to_datetime(data['date'].astype(str), format="%Y%m%d")
data['weekday'] = data['datestamp'].dt.weekday
data['weekfactor'] = data['weekday'].astype(str)

# Timeline calculations
data['timeline'] = np.where(data['group_year'] == 1, 
                           (data['datestamp'] - pd.Timestamp('2020-01-01')).dt.days, 
                           (data['datestamp'] - pd.Timestamp('2019-01-01')).dt.days)

# Define event and festival
date_conditions = [
    (data['date'].between(20200120, 20200123)),
    (data['date'].between(20200124, 20200223)),
    (data['date'].between(20200224, 20200508)),
    (data['date'] >= 20200509),
    (data['date'].between(20200210, 20200216)),
    (data['date'].between(20200217, 20200223)),
    (data['date'].between(20200321, 20200328)),
    (data['date'].between(20200329, 20200508))
]

event_columns = ['Tran', 'E1', 'E2', 'E3', 'R1', 'R2', 'Low', 'Open']
# Tran：人传人；E1：一级响应；E2：二级响应；E3：三级响应；R1：第一次复工；R2：第二次复工；Low：低风险；Open：公共空间开放

for col, cond in zip(event_columns, date_conditions):
    data[col] = np.where(cond, 1, 0)

# Define tp0 - tp7 columns（代表上述的8个不同时期，不重合）
data['tp0'] = np.where(data['date'].between(20200120, 20200123), 1, 0)
data['tp1'] = np.where(data['date'].between(20200124, 20200209), 1, 0)
data['tp2'] = np.where(data['date'].between(20200210, 20200216), 1, 0)
data['tp3'] = np.where(data['date'].between(20200217, 20200223), 1, 0)
data['tp4'] = np.where(data['date'].between(20200224, 20200320), 1, 0)
data['tp5'] = np.where(data['date'].between(20200321, 20200328), 1, 0)
data['tp6'] = np.where(data['date'].between(20200329, 20200508), 1, 0)
data['tp7'] = np.where(data['date'] >= 20200509, 1, 0)

# Define holidays columns
holidays_conditions = [
    (data['date'].isin([20190101, 20200101])),
    (data['date'].between(20190128, 20190204) | data['date'].between(20200117, 20200124)),
    (data['date'].between(20190204, 20190210) | data['date'].between(20200124, 20200202)),
    (data['date'].between(20190405, 20190407) | data['date'].between(20200404, 20200406)),
    (data['date'].between(20190501, 20190504) | data['date'].between(20200501, 20200505))
]

holidays_columns = ['newyear', 'sprtransp', 'spring', 'tomb', 'labor']
for col, cond in zip(holidays_columns, holidays_conditions):
    data[col] = np.where(cond, 1, 0)
    
# Joining the data and precip DataFrames
data = pd.merge(data, precip, on="date", how="inner")

# Calculating the quantiles for the income column
income_thre = data['income'].quantile(q=[0, 0.2, 0.4, 0.6, 0.8, 1])

# Updating and creating new columns
data['age'] = pd.cut(data['age'], bins=[0, 5, 13, 16, float('inf')], labels=['Young', 'Adult', 'Elder', 'Elder'], right=False,  ordered=False)
data['gender'] = data['gender'].map({1: 'Male', 2: 'Female'})
data['income'] = pd.cut(data['income'], bins=[0] + list(income_thre)[1:], labels=['Bottom', 'Lower', 'Middle', 'Upper', 'Superior'], right=False, include_lowest=True)

data['income3'] = data['income'].map({
    'Bottom': 'Bottom',
    'Superior': 'Upper',
    'Lower': 'Middle',
    'Middle': 'Middle',
    'Upper': 'Middle'
})

# Converting columns to category type with specific levels
data['age'] = data['age'].astype('category')
data['gender'] = data['gender'].astype('category')
data['income'] = pd.Categorical(data['income'], categories=["Bottom", "Lower", "Middle", "Upper", "Superior"], ordered=True)
data['income3'] = pd.Categorical(data['income3'], categories=["Middle", "Bottom", "Upper"], ordered=True)

gc.collect()

26

In [7]:
data.to_csv('data/demo12W_30_processed.csv',index=False)

In [9]:
print(data.isnull().sum())

pid                         0
date                        0
move_frequency              0
move_distance               0
trip_time                   0
home_time                   0
norm_sum              1848403
norm_mean               34600
norm_max              1848403
norm_home_activity    6086872
norm_home_base        2249310
group_year                  0
voluntary                   0
gender                      0
age                         0
income                   6060
datestamp                   0
weekday                     0
weekfactor                  0
timeline                    0
Tran                        0
E1                          0
E2                          0
E3                          0
R1                          0
R2                          0
Low                         0
Open                        0
tp0                         0
tp1                         0
tp2                         0
tp3                         0
tp4                         0
tp5       

# data sampling

In [67]:
data = pd.read_csv('data/demo12W_30_processed.csv')

In [68]:
# Sampling: select 5,000 individuals randomly
sample_it = data['pid'].drop_duplicates().sample(n=1000)
sample_data = data[data['pid'].isin(sample_it)]
print(sample_data.isnull().sum())

pid                       0
date                      0
move_frequency            0
move_distance             0
trip_time                 0
home_time                 0
norm_sum              14587
norm_mean               262
norm_max              14587
norm_home_activity    48398
norm_home_base        17832
group_year                0
voluntary                 0
gender                    0
age                       0
income                    0
datestamp                 0
weekday                   0
weekfactor                0
timeline                  0
Tran                      0
E1                        0
E2                        0
E3                        0
R1                        0
R2                        0
Low                       0
Open                      0
tp0                       0
tp1                       0
tp2                       0
tp3                       0
tp4                       0
tp5                       0
tp6                       0
tp7                 

In [69]:
del data
gc.collect()

0

In [70]:
sample_data = sample_data.sort_values(by=['pid', 'date']).reset_index(drop=True)

In [71]:
sample_data.head()

,pid,date,move_frequency,move_distance,trip_time,home_time,norm_sum,norm_mean,norm_max,norm_home_activity,...,tp5,tp6,tp7,newyear,sprtransp,spring,tomb,labor,precip,income3
0,19385,20190101,2.0,5678.0,362.0,8.046,37009.366667,1417.983,2481.0,21280.467,...,0,0,0,1,0,0,0,0,0.0,Upper
1,19385,20190102,6.0,36184.0,4302.0,12.568,65556.966667,1465.506,2775.0,20361.467,...,0,0,0,0,0,0,0,0,0.1,Upper
2,19385,20190103,4.0,59256.0,4683.0,11.998,79392.700000,2074.722,7558.0,26336.833,...,0,0,0,0,0,0,0,0,0.0,Upper
3,19385,20190104,2.0,22218.0,1751.0,3.229,59087.200000,2557.887,2891.0,13800.533,...,0,0,0,0,0,0,0,0,0.1,Upper
4,19385,20190105,4.0,30634.0,4301.0,5.537,20674.266667,1125.641,1909.0,15378.967,...,0,0,0,0,0,0,0,0,0.0,Upper


In [95]:
# use yesterday norm_mean as conformity factor
sample_data['conformity'] = sample_data.groupby('pid')['norm_mean'].shift(1)

# use data in 2020
sample_data2020 = sample_data.query('date>=20200101').reset_index(drop=True)

# fill null in conformity
sample_data2020['conformity'] = sample_data2020['conformity'].fillna(0)

# encoding categories
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
ind_attr_cols = ['gender', 'age', 'income', 'income3']
for col in ind_attr_cols:
    sample_data2020[col] = le.fit_transform(sample_data2020[col])
sample_data2020['income3'] = sample_data2020['income3'].fillna(0)

# weekday one-hot encoding
one_hot_encoded = pd.get_dummies(sample_data2020['weekday'], prefix='weekday')
sample_data2020 = sample_data2020.drop('weekday', axis=1)
sample_data2020 = pd.concat([sample_data2020, one_hot_encoded], axis=1)
# sample_data2020.head()

# use is_workday
# from chinese_calendar import is_workday
# sample_data2020['datestamp'] = pd.to_datetime(sample_data2020['date'].astype(str), format="%Y%m%d")
# sample_data2020['workday'] = sample_data2020['datestamp'].apply(lambda x: int(is_workday(x)))

In [96]:
# construct policy features
sample_data2020['restrict'] = np.where(sample_data2020['date'].between(20200124, 20200223), 1, 0)
sample_data2020['open'] = np.where(sample_data2020['date'] > 20200223, 1, 0)

In [57]:
sample_data2020.head(100)

,pid,date,move_frequency,move_distance,trip_time,home_time,norm_sum,norm_mean,norm_max,norm_home_activity,...,sprtransp,spring,tomb,labor,precip,income3,conformity,workday,restrict,open
0,1226,20200101,4.0,14772.0,2571.0,13.343,36834.900000,994.642,1919.0,20808.200,...,0,0,0,0,0.0,0,1098.701,0,0,0
1,1226,20200102,2.0,24070.0,4109.0,5.616,46951.633333,1453.611,2056.0,7661.633,...,0,0,0,0,0.0,0,994.642,1,0,0
2,1226,20200103,2.0,24070.0,3779.0,6.646,43883.333333,1435.660,2089.0,9424.767,...,0,0,0,0,0.0,0,1453.611,1,0,0
3,1226,20200104,4.0,27946.0,2968.0,10.926,48074.266667,1073.885,1756.0,13098.933,...,0,0,0,0,0.0,0,1435.660,0,0,0
4,1226,20200105,0.0,0.0,0.0,24.000,33139.733333,715.246,888.0,33139.733,...,0,0,0,0,0.0,0,1073.885,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1226,20200405,0.0,0.0,0.0,24.000,28503.733333,602.616,728.0,28503.733,...,0,0,1,0,10.2,0,640.313,0,0,1
96,1226,20200406,0.0,0.0,0.0,24.000,16295.533333,577.174,661.0,16295.533,...,0,0,1,0,41.9,0,602.616,0,0,1
97,1226,20200407,2.0,24076.0,3815.0,2.759,41547.200000,1363.694,1824.0,2302.067,...,0,0,0,0,0.2,0,577.174,1,0,1
98,1226,20200408,2.0,24076.0,4316.0,9.978,46432.400000,1034.129,1885.0,6821.200,...,0,0,0,0,0.0,0,1363.694,1,0,1


In [13]:
print(len(sample_data2020)/len(sample_data2020['pid'].unique()))

152.0


In [25]:
print(sample_data2020.isnull().sum())

pid                       0
date                      0
move_frequency            0
move_distance             0
trip_time                 0
home_time                 0
norm_sum              30103
norm_mean               165
norm_max              30103
norm_home_activity    91076
norm_home_base        36144
group_year                0
voluntary                 0
gender                    0
age                       0
income                    0
datestamp                 0
weekfactor                0
timeline                  0
Tran                      0
E1                        0
E2                        0
E3                        0
R1                        0
R2                        0
Low                       0
Open                      0
tp0                       0
tp1                       0
tp2                       0
tp3                       0
tp4                       0
tp5                       0
tp6                       0
tp7                       0
newyear             

# data transformation

covariates：gender, age, income, festivals(newyear, sprtransp, spring, tomb, labor), events(Tran), weekday  
treatments: norm_mean, policy(如何编码？), voluntary(=case)  
outcome: move_distance  
Tran：人传人；E1：一级响应；E2：二级响应；E3：三级响应；R1：第一次复工；R2：第二次复工；Low：低风险；Open：公共空间开放

In [100]:
def normalize_dataset(dataset, num_covariates, num_treatments):

        for covariate_id in range(num_covariates):
            covariate_mean = np.mean(dataset['previous_covariates'][:, :, covariate_id])
            covariate_std = np.std(dataset['previous_covariates'][:, :, covariate_id])
            dataset['previous_covariates'][:, :, covariate_id] = (dataset['previous_covariates'][:, :,
                                                      covariate_id] - covariate_mean) / covariate_std
            # covariate_max = np.max(dataset['previous_covariates'][:, :, covariate_id])
            # covariate_min = np.min(dataset['previous_covariates'][:, :, covariate_id])
            # dataset['previous_covariates'][:, :, covariate_id] = (dataset['previous_covariates'][:, :,
            #                                           covariate_id] - covariate_min) / (covariate_max - covariate_min)

        for covariate_id in range(num_covariates):
            covariate_mean = np.mean(dataset['covariates'][:, :, covariate_id])
            covariate_std = np.std(dataset['covariates'][:, :, covariate_id])
            dataset['covariates'][:, :, covariate_id] = (dataset['covariates'][:, :,
                                                      covariate_id] - covariate_mean) / covariate_std
            # covariate_max = np.max(dataset['covariates'][:, :, covariate_id])
            # covariate_min = np.min(dataset['covariates'][:, :, covariate_id])
            # dataset['covariates'][:, :, covariate_id] = (dataset['covariates'][:, :,
            #                                           covariate_id] - covariate_min) / (covariate_max - covariate_min)
        
        for treatment_id in range(num_treatments):
            treatment_mean = np.mean(dataset['previous_treatments'][:, :, treatment_id])
            treatment_std = np.std(dataset['previous_treatments'][:, :, treatment_id])
            dataset['previous_treatments'][:, :, treatment_id] = (dataset['previous_treatments'][:, :,
                                                      treatment_id] - treatment_mean) / treatment_std
            # use Min-Max scaling
            # treatment_max = np.max(dataset['previous_treatments'][:, :, treatment_id])
            # treatment_min = np.min(dataset['previous_treatments'][:, :, treatment_id])
            # dataset['previous_treatments'][:, :, treatment_id] = (dataset['previous_treatments'][:, :,
            #                                           treatment_id] - treatment_min) / (treatment_max - treatment_min)

        for treatment_id in range(num_treatments):
            treatment_mean = np.mean(dataset['treatments'][:, :, treatment_id])
            treatment_std = np.std(dataset['treatments'][:, :, treatment_id])
            dataset['treatments'][:, :, treatment_id] = (dataset['treatments'][:, :,
                                                      treatment_id] - treatment_mean) / treatment_std
            # use Min-Max scaling
            # treatment_max = np.max(dataset['treatments'][:, :, treatment_id])
            # treatment_min = np.min(dataset['treatments'][:, :, treatment_id])
            # dataset['treatments'][:, :, treatment_id] = (dataset['treatments'][:, :,
            #                                           treatment_id] - treatment_min) / (treatment_max - treatment_min)

        outcome_mean = np.mean(dataset['outcomes'])
        outcome_std= np.std(dataset['outcomes'])
        dataset['outcomes'] = (dataset['outcomes'] - outcome_mean) /outcome_std

        return dataset

def generate_dataset(data, timesteps, covariate_cols, treatment_cols, outcome_col):
            dataset = dict()

            dataset['previous_covariates'] = []
            dataset['previous_treatments'] = []
            dataset['covariates'] = []
            dataset['treatments'] = []
            dataset['sequence_length'] = []
            dataset['outcomes'] = []
            
            for pid in data['pid'].unique():
                single_data = data[data['pid']==pid].reset_index(drop=True)
                covariates_history = single_data[covariate_cols].values
                treatments_history = single_data[treatment_cols].values
                
                previous_covariates = np.array(covariates_history[1:timesteps - 1])
                previous_treatments = np.array(np.array(treatments_history[1:timesteps - 1]))
                
                covariates = np.array(covariates_history[1:timesteps])
                treatments = np.array(treatments_history[1:timesteps])
                
                outcomes = single_data[outcome_col].values[1:timesteps]
                outcomes = outcomes[:, np.newaxis]
                
                dataset['previous_covariates'].append(np.array(previous_covariates))
                dataset['previous_treatments'].append(np.array(previous_treatments))
                dataset['covariates'].append(np.array(covariates))
                dataset['treatments'].append(np.array(treatments))
                dataset['sequence_length'].append(np.array(timesteps))
                dataset['outcomes'].append(np.array(outcomes))
            
            for key in dataset.keys():
                dataset[key] = np.array(dataset[key])

            return dataset

In [101]:
timesteps = int(len(sample_data2020)/len(sample_data2020['pid'].unique()))
covariate_cols = ['gender','age','income3','weekday_0','weekday_1','weekday_2','weekday_3','weekday_4', 'weekday_5', 'weekday_6',
                  'sprtransp','spring','precip', 'voluntary']
treatment_cols = ['conformity','restrict','open']
# treatment_cols = ['restrict','open']
outcome_col = 'move_distance'
dataset = generate_dataset(sample_data2020, timesteps, covariate_cols, treatment_cols, outcome_col)
norm_dataset = normalize_dataset(dataset, len(covariate_cols), len(treatment_cols)-2) # treatment仅正则化conformity和voluntary两项

In [83]:
print(norm_dataset)

{'previous_covariates': array([[[ 0.65309534, -0.13111262,  1.62996707, ..., -0.26726124,
         -0.3273241 , -0.3660031 ],
        [ 0.65309534, -0.13111262,  1.62996707, ..., -0.26726124,
         -0.3273241 , -0.3660031 ],
        [ 0.65309534, -0.13111262,  1.62996707, ..., -0.26726124,
         -0.3273241 , -0.3660031 ],
        ...,
        [ 0.65309534, -0.13111262,  1.62996707, ..., -0.26726124,
         -0.30532651, -0.3660031 ],
        [ 0.65309534, -0.13111262,  1.62996707, ..., -0.26726124,
         -0.3273241 , -0.3660031 ],
        [ 0.65309534, -0.13111262,  1.62996707, ..., -0.26726124,
          5.00709084, -0.3660031 ]],

       [[ 0.65309534, -0.13111262,  1.62996707, ..., -0.26726124,
         -0.3273241 , -0.3660031 ],
        [ 0.65309534, -0.13111262,  1.62996707, ..., -0.26726124,
         -0.3273241 , -0.3660031 ],
        [ 0.65309534, -0.13111262,  1.62996707, ..., -0.26726124,
         -0.3273241 , -0.3660031 ],
        ...,
        [ 0.65309534, -0.13111

In [84]:
for key in list(dataset.keys()):
    print(key)
    print(dataset[key].shape)

previous_covariates
(1000, 150, 8)
previous_treatments
(1000, 150, 3)
covariates
(1000, 151, 8)
treatments
(1000, 151, 3)
sequence_length
(1000,)
outcomes
(1000, 151, 1)


In [102]:
# export data
import pickle

def write_results_to_file(filename, data):
    with open(filename, 'wb') as handle:
        pickle.dump(data, handle, protocol=2)

write_results_to_file('data/real_data_norm_sample1000_weekday.txt', norm_dataset)
#write_results_to_file('data/real_data_sample1000.txt', dataset)

In [79]:
features = sample_data2020[covariate_cols+treatment_cols+[outcome_col]]
print(features.isna().sum())

gender           0
age              0
income3          0
workday          0
sprtransp        0
spring           0
precip           0
voluntary        0
conformity       0
restrict         0
open             0
move_distance    0
dtype: int64


In [46]:
iptw = np.load("results/rmsn_sample_10000_gamma_0.6_use_confounders_False/propensity_scores.npy")
np.any(np.isnan(iptw))

False

In [49]:
print(iptw.min())

-0.27508382828776695


In [41]:
outputs = norm_dataset['outcomes']
sequence_lengths = norm_dataset['sequence_length']
horizon = 1
b_predict_actions = False
active_entries = np.zeros(outputs.shape)
for i in range(sequence_lengths.shape[0]):
    sequence_length = int(sequence_lengths[i])

    if not b_predict_actions:
        for k in range(horizon):
            #include the censoring point too, but ignore future shifts that don't exist
            active_entries[i, :sequence_length-k, k] = 1
    else:
        active_entries[i, :sequence_length, :] = 1

In [43]:
np.sum(active_entries)

1510000.0

In [62]:
def load_data_from_file(filename):
    with open(filename, 'rb') as handle:
        return pickle.load(handle)

# 使用函数加载数据
filename = "results/simulated_data/_dataset_with_substitute_confounders.txt"  # 更改为您的文件名
simulated_data = load_data_from_file(filename)
print(simulated_data)

{'previous_covariates': array([[[ 0.4836432 ,  0.14829019, -0.27194955],
        [ 0.21966952,  0.159632  , -0.29473256],
        [ 0.31210715,  0.23716383, -0.18455728],
        ...,
        [-0.01889956, -0.01106907,  0.01809696],
        [-0.02130581,  0.00441948,  0.01511471],
        [ 0.        ,  0.        ,  0.        ]],

       [[-0.10181909, -0.16746599,  0.65813375],
        [-0.27828921, -0.08667827,  0.841197  ],
        [-0.37494351,  0.03567515,  0.61385476],
        ...,
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]],

       [[ 0.34463483, -0.25486716, -0.46962224],
        [ 0.10949454, -0.14256839, -0.42181919],
        [ 0.17931144, -0.05347157, -0.10505994],
        ...,
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]],

       ...,

       [[-0.28026265,  0.50328932,  

In [85]:
for key in list(simulated_data.keys()):
    print(key)
    print(simulated_data[key].shape)

previous_covariates
(5000, 29, 3)
previous_treatments
(5000, 29, 3)
covariates
(5000, 30, 3)
confounders
(5000, 30, 1)
treatments
(5000, 30, 3)
sequence_length
(5000,)
outcomes
(5000, 30, 1)


In [91]:
print(len(dataset['outcomes'][1,:,:]))
print(dataset['sequence_length'][1])

151
152
